In [ ]:
import datetime
import os
import numpy as np
from skimage import io
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage import img_as_ubyte, img_as_float
from scipy import ndimage 
from matplotlib import pyplot as plt
from matplotlib.colors import hsv_to_rgb
import cv2
from PIL import Image
from skimage.color import rgb2gray
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC 
from sklearn.naive_bayes import GaussianNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV 
import pickle
import skimage.data as data
import skimage.segmentation as seg
import skimage.filters as filters
import skimage.draw as draw
import skimage.color as color
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float
from skimage.exposure import rescale_intensity

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
label_list=[]
image_list=[]
path1= 'C:\\Users\\Shashank\\Python Projects\\Plant Disease\\RiceDiseases-DataSet-master'

listing=os.listdir(path1)
for file in listing:
    
    list2=os.listdir(path1+'\\'+file)
    for image in list2:
        im=cv2.imread(path1+'\\'+file+'\\'+image)
        
        
        
        res_img = segmentation(im)
        

        
        image_list.append(res_img)
        label_list.append(file)

svc(label_list,image_list)

In [ ]:
def segmentation(im, mode=0):
    
    gray = None
    img_hsv = np.zeros((256,256,3))
    if mode==0:
        gray = cv2.cvtColor(im,cv2.COLOR_RGB2GRAY)
    

    #gray = rgb2gray(im)
    
    #Thresholding
    elif mode==1:
        img_hsv = cv2.cvtColor(np.float32(im),cv2.COLOR_BGR2HSV)
        gray = rgb2gray(img_hsv)
    

    
    
        gray_r = gray.reshape(gray.shape[0]*gray.shape[1])
        for i in range(gray_r.shape[0]):
            if gray_r[i] > gray_r.mean():
                gray_r[i] = 1
            else:
                gray_r[i] = 0
        gray = gray_r.reshape(gray.shape[0],gray.shape[1])

        gray = rgb2gray(im)
        gray_r = gray.reshape(gray.shape[0]*gray.shape[1])
        for i in range(gray_r.shape[0]):
            if gray_r[i] > 0.8 :
                gray_r[i] = 255
            elif gray_r[i] > 0.6:
                gray_r[i] = 128

            else:
                gray_r[i] = 0
        gray = gray_r.reshape(gray.shape[0],gray.shape[1])
    
    
    #Felzenszwalb Segmentation
    elif mode==2:
    
        image_felzenszwalb = seg.felzenszwalb(img_hsv)
        gray = cv2.cvtColor(image_felzenszwalb,cv2.COLOR_RGB2GRAY)
    
    # SLIC Segmentation
    elif mode==3:
    
        vis = np.zeros(im.shape[:2], dtype="float")

        segments = slic(img_as_float(im), n_segments=200,slic_zero=True)
        for v in np.unique(segments):
            mask = np.ones(im.shape[:2])
            mask[segments == v] = 0
            C = segment_colorfulness(im, mask)
            vis[segments == v] = C
        vis = rescale_intensity(vis, out_range=(0, 255)).astype("uint8")
        alpha = 0.6
        overlay = np.dstack([vis] * 3)
        output = im.copy()
        cv2.addWeighted(overlay, alpha, output, 1 - alpha, 0, output)
        gray = cv2.cvtColor(output,cv2.COLOR_BGR2GRAY)
    
    return gray

In [ ]:
def svc(label_list,image_list):
    label_enc=LabelEncoder()
    image_labels=label_enc.fit_transform(label_list)

    n_classes = len(label_enc.classes_)
    XTrain,XTest,YTrain,YTest = train_test_split(image_list,image_labels,test_size=0.2)
    XTrain = np.array(XTrain)
    nsamples, nx, ny = XTrain.shape
    XTrainRed = XTrain.reshape((nsamples,nx*ny))
    XTest = np.array(XTest)
    nsamples, nx, ny = XTest.shape
    XTestRed = XTest.reshape((nsamples,nx*ny))

    model = OneVsRestClassifier(SVC())


    parameters = {
        "estimator__C": [1,2,4,8],
        "estimator__kernel": ["poly","rbf"],
        "estimator__degree":[1, 2, 3, 4],
    } 

    model = GridSearchCV(model, parameters, refit = True, verbose = 3)
    model.fit(XTrainRed, YTrain)
    #model = SVC(kernel = 'rbf', C = 1).fit(XTrainRed, YTrain) 
    svm_predictions = model.predict(XTestRed) 
    #model = GaussianNB()
    #model.fit(XTrainRed,YTrain)
    accuracy = model.score(XTestRed, YTest)
    #accuracy

    print(accuracy)
    pickle.dump(model,open('SVM61p.sav','wb'))



